- Programmer: Chen Luo, Project Chief: Yang Li


- Date: 04/09/2019

> 要求：
>
>- 将总体微博语料按照三个时间段进行拆分：09-12；13-15；16-18
>
>
>- 输出结果：词频统计、语义网络（基于词语共现矩阵）
>
>
>- 网络中的节点尽量少一些，思考一个合理的筛选方法

In [67]:
from IPython.display import HTML

HTML('/Users/lyndon/AnacondaProjects/Style.css')

### 时间解析

In [3]:
import re
import pandas as pd

In [4]:
# 读入数据
gmo1 = pd.read_excel('./GMOData.xlsx', sheet_name='Sheet1', header=0)
gmo1.head()

,Unnamed: 0,_id_x,nick_name,gender,province,city,labrel,birthday,register time,tweets_num,...,person_url,_id_y,user_id,created_at,content.1,like_num,repost_num,comment_num,content_cutted,topic
0,0,2.054992e+09,潇潇diana,女,北京,海淀区,NaN,01-01,NaN,891.0,...,http://weibo.com/dianahxw,2054992375_Grif48Q52,2.054992e+09,3分钟前,《一个妈妈一天的心路历程》《吃饭篇》牛奶有无三聚氰胺超标，会不会喝成大头？面包有没有添加剂？...,0.0,0.0,0.0,纯棉 苏丹红 衣服 家里 婴幼儿 琼胶 质地 喇叭 制成 医生 生病 床单 蔬菜 专用 吃饭...,12
1,1,1.773917e+09,睡不饱的任镳,男,上海,杨浦区,NaN,1986-05-21,40389.0,1353.0,...,http://weibo.com/u/1773916610,1773916610_GrieZ88DV,1.773917e+09,3分钟前,我发现现在的媒体，有了微博后，关注度会大幅度增加，快速传播，影响很大，不过有个缺点，就是不够...,0.0,0.0,0.0,天津 国内 顶尖 喜好 缺点 添上 崔 不伦不类 专业性 花荣 有名 股侠 传播 缺陷 快速...,19
2,2,1.773917e+09,睡不饱的任镳,男,上海,杨浦区,NaN,1986-05-21,NaN,1353.0,...,http://weibo.com/u/1773916610,1773916610_Acvjj1CiK,1.773917e+09,2013-10-04 22:12:09,院士：中国没有拒绝转基因技术的资本】据工程院院士吴孔明分析，我国粮食产需间矛盾突出，已经不允...,0.0,0.0,0.0,详见 发展 粮食 作物 zR2wIgX 危险 吴 棉花 搁置 院士 资本 放弃 技术 拒绝 ...,5
3,3,3.021199e+09,狙击手蝈蝈,男,广东,东莞,NaN,NaN,NaN,3329.0,...,http://weibo.com/u/3021199493,3021199493_Gribg5cCN,3.021199e+09,13分钟前,铁证如山！日军性暴行受害者两姐妹证言公布】为救其他人，时年14岁的彭仁寿被强征为“慰安妇”，...,0.0,0.0,1.0,强征 救 日军 彭 传递 彭竹英 刀疤 触目 慰安妇 终生 细菌武器 魔爪 未能 逃脱 受害...,8
4,4,3.021199e+09,狙击手蝈蝈,男,广东,东莞,NaN,NaN,NaN,3329.0,...,http://weibo.com/u/3021199493,3021199493_EmYJuh1za,3.021199e+09,2016-12-20 14:26:32,黑龙江省明年5月起全面禁止转基因作物！】根据最新修订的《黑龙江省食品安全条例》，自2017年...,0.0,1.0,1.0,含有 RIcPEQm 销售 作物 粮食产量 食品安全 条例 大省 加工 大豆 全面禁止 明年...,3


In [5]:
# 数据行列数
print('%s 行 %s 列'%(gmo1.shape[0], gmo1.shape[1]))

886837 行 25 列


In [6]:
# 抽取内容列、日期列
gmo2 = gmo1[['content.1', 'created_at']]
gmo2.shape

(886837, 2)

In [8]:
# 数据备份
gmo2.to_excel('./v2/gmo.xlsx', index=None)

In [9]:
# 数据复制
gmo3 = gmo2.copy()

# 抽取年份
pattern_match = re.compile('(\d{4}-\d{2}-\d{2})')
def getYear(s):
    yearMatch = pattern_match.search(str(s))
    if yearMatch != None:
        year = yearMatch.group(1).split('-')[0].strip()
    else:
        year = '2018'
    return year

gmo3['year'] = gmo3['created_at'].apply(getYear)
gmo3.head()

,content.1,created_at,year
0,《一个妈妈一天的心路历程》《吃饭篇》牛奶有无三聚氰胺超标，会不会喝成大头？面包有没有添加剂？...,3分钟前,2018
1,我发现现在的媒体，有了微博后，关注度会大幅度增加，快速传播，影响很大，不过有个缺点，就是不够...,3分钟前,2018
2,院士：中国没有拒绝转基因技术的资本】据工程院院士吴孔明分析，我国粮食产需间矛盾突出，已经不允...,2013-10-04 22:12:09,2013
3,铁证如山！日军性暴行受害者两姐妹证言公布】为救其他人，时年14岁的彭仁寿被强征为“慰安妇”，...,13分钟前,2018
4,黑龙江省明年5月起全面禁止转基因作物！】根据最新修订的《黑龙江省食品安全条例》，自2017年...,2016-12-20 14:26:32,2016


In [10]:
# 微博数量按年份统计
groupYear = gmo3.groupby('year')
groupYear.size()

year
2009       145
2010     10997
2011     47428
2012    115129
2013    142112
2014    140611
2015    106531
2016    132339
2017    121113
2018     70432
dtype: int64

In [14]:
# 微博总数量
print(len(gmo3['year']))

886837


### 抽样

In [16]:
# 09-12年微博总量：173699
gmo_09_12 = gmo3[gmo3['year'].isin(['2009', '2010', '2011', '2012'])]
print(gmo_09_12.shape)

sample_09_12 = gmo_09_12.sample(n=20000, random_state=2009)
print(sample_09_12.shape)

sample_09_12.to_excel('./v2/09_12.xlsx', index=None)

(173699, 3)
(20000, 3)


In [17]:
# 13-15年微博总量：389254
gmo_13_15 = gmo3[gmo3['year'].isin(['2013', '2014', '2015'])]
print(gmo_13_15.shape)

sample_13_15 = gmo_13_15.sample(n=20000, random_state=2013)
print(sample_13_15.shape)

sample_13_15.to_excel('./v2/13_15.xlsx', index=None)

(389254, 3)
(20000, 3)


In [19]:
# 16-18年微博总量：323884
gmo_16_18 = gmo3[gmo3['year'].isin(['2016', '2017', '2018'])]
print(gmo_16_18.shape)

sample_16_18 = gmo_16_18.sample(n=20000, random_state=2016)
print(sample_16_18.shape)

sample_16_18.to_excel('./v2/16_18.xlsx', index=None)

(323884, 3)
(20000, 3)


### 分词、统计词频

In [35]:
import os
import codecs
import jieba
import jieba.posseg as pseg
from multiprocessing import Pool
from collections import defaultdict

In [21]:
# 读取自定义词典
jieba.load_userdict('./v2/userdict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_6/73bms9_11733g1hbd2_jmrzc0000gn/T/jieba.cache
Loading model cost 0.778 seconds.
Prefix dict has been built succesfully.


In [22]:
# 导入停用词
stopwords = list()
with open('./v2/stopwords.txt', 'r') as s:
    for word in s.readlines():
        stopwords.append(word.strip())
stopwords = list(set(stopwords))
print(len(stopwords))

2297


In [41]:
# 每次返回2000条数据
def loadData(df):
    trunk = 2000
    idx = 0
    contents = list()
    for line in df['content.1']:
        contents.append(str(line).strip())
        idx += 1
        if idx % trunk == 0:
            yield contents
            contents = list()

In [42]:
# 分词函数，仅保留名词、形容词、动词
# 词性参考 https://blog.csdn.net/suibianshen2012/article/details/53487157
def tokenize(contents):
    result = list()
    for content in contents:
        _ = list()
        for word, flag in pseg.cut(content):
            if (word not in stopwords) and (len(word) > 1) and (re.search(r'^n|a|v', flag) != None):
                _.append(word.strip())
        result.append(' '.join(_))
    return result

In [43]:
# 并行分词
def parTokenize(file, df):
    target = codecs.open('./v2/' + file.split('/')[-1].split('.')[0] + '_seg.txt', 'w', 'utf-8')
    contents = loadData(df)
    cpus = 5
    ichunk = 0
    for content in contents:
        pool = Pool(cpus)
        step = int(len(content) / cpus)
        temp = [content[i:i+step] for i in range(0, len(content), step)]
        results = pool.map(tokenize, temp)
        pool.close()
        pool.join()
        for r in results:
            for i in r:
                target.write(i + '\n')
        ichunk += 1
        print('Progress:', len(content)*ichunk)
    target.close()

In [46]:
# 执行并行分词
meta_path = './v2/'
for file in os.listdir('./v2'):
    if re.search(r'(\d{2}_\d{2}.xlsx)', file):
        parTokenize(meta_path + file, pd.read_excel(meta_path + file, header=0))
        print(file, 'finished.', '\n')

Progress: 2000
Progress: 4000
Progress: 6000
Progress: 8000
Progress: 10000
Progress: 12000
Progress: 14000
Progress: 16000
Progress: 18000
Progress: 20000
09_12.xlsx finished. 

.DS_Store finished. 

Progress: 2000
Progress: 4000
Progress: 6000
Progress: 8000
Progress: 10000
Progress: 12000
Progress: 14000
Progress: 16000
Progress: 18000
Progress: 20000
13_15.xlsx finished. 

userdict.txt finished. 

stopwords.txt finished. 

gmo.xlsx finished. 

Progress: 2000
Progress: 4000
Progress: 6000
Progress: 8000
Progress: 10000
Progress: 12000
Progress: 14000
Progress: 16000
Progress: 18000
Progress: 20000
16_18.xlsx finished. 



In [47]:
# 统计词频
def freq(file):
    frequency = defaultdict(int)
    target = codecs.open('./v2/' + file.split('/')[-1].split('.')[0].replace('seg', 'freq') + '.txt', 'w', 'utf-8')
    source = codecs.open(file, 'r', 'utf-8')
    for line in source.readlines():
        for word in line.split():
            frequency[word] += 1
    for w, f in sorted(dict(frequency).items(), key=lambda x: x[1], reverse=True):
        target.write(w + ' ' + str(f) + '\n')

for file in os.listdir('./v2/'):
    if re.search(r'\d{2}_\d{2}_seg.txt', file):
        freq('./v2/' + file)

### 构建网络
- 需要排除搜索词“转基因”（dominant word），避免对后续网络分析造成干扰

In [48]:
from itertools import combinations

In [72]:
# 每一行的分词需要去重，再统计共现关系
def co(file):
    co_occur = defaultdict(int)
    # 排除“转基因”的文件名中包含refine
    target = codecs.open('./v2/' + file.split('/')[-1].split('.')[0].replace('seg', 'network_refine') + '.csv', 'w', 'utf-8')
    source = codecs.open(file, 'r', 'utf-8')
    for line in source.readlines():
        new_line = list(set(line.split()))
        new_line.sort()
        for pair in combinations(new_line, 2):
            if '转基因' not in pair:
                co_occur[pair] += 1
    for p, f in sorted(dict(co_occur).items(), key=lambda x: x[1], reverse=True):
        target.write(p[0] + ',' + p[1] + ',' + str(f) + '\n')

for file in os.listdir('./v2/'):
    if re.search(r'\d{2}_\d{2}_seg.txt', file):
        co('./v2/' + file)

In [71]:
# try
l = [12, 23, 34]
for i in combinations(l, 2):
    print(i, i[0], type(i), 12 in i)

(12, 23) 12 <class 'tuple'> True
(12, 34) 12 <class 'tuple'> True
(23, 34) 23 <class 'tuple'> False


### 选择呈现节点
- 先按照频数从高到低排序，然后设定比率

In [74]:
for file in os.listdir('./v2/'):
    # 用 *_refine.csv 文件
    if re.search(r'\d{2}_\d{2}_network_refine.csv', file):
        df = pd.read_csv('./v2/' + file, sep=',', header=None)
        frequency = list(set(list(df[2])))
        frequency.sort(reverse=True)
        print(frequency, '\n')
        print('文件名：', file, '\n', '频次单独值数量：', len(frequency), '\n', '截尾值：', frequency[round(len(frequency) * 0.25) + 1], '\n\n\n')

[1082, 556, 555, 540, 477, 439, 416, 380, 379, 365, 360, 346, 323, 320, 315, 309, 308, 304, 299, 281, 280, 274, 272, 271, 270, 261, 260, 258, 256, 248, 247, 245, 242, 240, 233, 229, 226, 224, 223, 222, 220, 216, 214, 205, 204, 202, 201, 200, 199, 195, 189, 188, 186, 183, 181, 180, 179, 178, 177, 176, 175, 174, 173, 170, 169, 166, 165, 164, 160, 159, 158, 157, 156, 155, 153, 152, 151, 149, 147, 145, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 110, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1] 


### Gephi绘图
- 节点重要性：特征向量中心度


- 模块化处理（目的：社区侦测；注意计算细节，如：使用权重），不同颜色渲染节点


- 语义网络核心文献